### Import Libraries



- To run this script we need the libraries below
- Pandas : For creating DataFrames
- Numpy:  For computations 
- OS : For modifying working directories 
- censusdata: This is a library for accessing US Census Bureau, we use some of its function to preview variables *
- Census: Main library for accessing the US Census Bureau API *
- pd.set_option allows us to view our entire output instead of it being truncated in jupyter notebook



 * You need to install these packages before you can use. Census = pip install census | censusdata = pip install CensusData

In [1]:
import pandas as pd
import numpy as np
import os
import censusdata as cs
from census import Census
import warnings

pd.set_option('display.max_rows',None, 'display.max_columns',None,'display.max_colwidth', None)

warnings.filterwarnings("ignore", message=".*DataFrame is highly fragmented.*")

In [2]:
pwd

'C:\\Users\\Thepr\\Downloads'

##### Pass your API Key

- Go to this [website](https://api.census.gov/data/key_signup.html 'sign up for your personal API key') and sign up your personal API key 

- Once you get your API key, call the Census function and pass your API key. Eg. Census("API Key") 

In [3]:
c = Census("e0577a26a616f4dda60446eae987e3b6d0d944a3")

#### Preview ACS variable 

In [4]:
cs.printtable(cs.censustable('acs5',2019,'B03002'))

Variable     | Table                          | Label                                                    | Type 
-------------------------------------------------------------------------------------------------------------------
B03002_001E  | HISPANIC OR LATINO ORIGIN BY R | !! Estimate Total:                                       | int  
B03002_002E  | HISPANIC OR LATINO ORIGIN BY R | !! !! Estimate Total: Not Hispanic or Latino:            | int  
B03002_003E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: White a | int  
B03002_004E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Black o | int  
B03002_005E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: America | int  
B03002_006E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Latino: Asian a | int  
B03002_007E  | HISPANIC OR LATINO ORIGIN BY R | !! !! !! Estimate Total: Not Hispanic or Lati

### ACS 5-Year Estimates


- **Race and Ethnicty**

In [12]:
import pandas as pd

Years = [2023]
AllData = []


variables = {
    'B03002_001E': 'Total Population',
    'B03002_003E': 'White Alone, Not Hispanic',
    'B03002_004E': 'Black or African American Alone',
    'B03002_006E': 'Asian Alone',
    'B03002_007E': 'Pacific Islander Alone',
    'B03002_008E': 'Some Other Race Alone',
    'B03002_009E': 'Two or More Races',
    'B03002_012E': 'Hispanic or Latino'
}

for year in Years:
    ACS = c.acs5.get(list(variables.keys()), {'for': 'place:04000', 'in': 'state:48'}, year=year)
    
    Data = pd.DataFrame(ACS)
    Data['year'] = year


    Data.rename(columns=variables, inplace=True)


    for col in variables.values():
        Data[col] = Data[col].astype(int)

    # Combine relevant categories
    Data['White'] = Data['White Alone, Not Hispanic']
    Data['Black'] = Data['Black or African American Alone']
    Data['Asian or Pacific Islander'] = Data['Asian Alone'] + Data['Pacific Islander Alone']
    Data['Other'] = Data['Some Other Race Alone'] + Data['Two or More Races']
    Data['Hispanic or Latino'] = Data['Hispanic or Latino']

    total = Data['Total Population'].iloc[0]

 
    rows = [
        ("White", Data['White'].iloc[0], round(Data['White'].iloc[0] / total * 100, 2)),
        ("Black", Data['Black'].iloc[0], round(Data['Black'].iloc[0] / total * 100, 2)),
        ("Asian or Pacific Islander", Data['Asian or Pacific Islander'].iloc[0], round(Data['Asian or Pacific Islander'].iloc[0] / total * 100, 2)),
        ("Other", Data['Other'].iloc[0], round(Data['Other'].iloc[0] / total * 100, 2)),
        ("Hispanic or Latino", Data['Hispanic or Latino'].iloc[0], round(Data['Hispanic or Latino'].iloc[0] / total * 100, 2)),
    ]

    EthnicityData = pd.DataFrame(rows, columns=['Ethnicity', 'Ethnicity/Race Total', 'Percent Ethnicity/Race'])
    AllData.append(EthnicityData)


FinalData = pd.concat(AllData, axis=0).reset_index(drop=True)
FinalData.to_csv('Arlington Ethnicity 5-Year Estimate ACS.csv')
FinalData


,Ethnicity,Ethnicity/Race Total,Percent Ethnicity/Race
0,White,141166,35.76
1,Black,86497,21.91
2,Asian or Pacific Islander,30004,7.60
3,Other,14189,3.59
4,Hispanic or Latino,122467,31.02


#### Veterans

#### Veterans